In [2]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact

In [13]:
sheet_id = '1aqHnDaiRXntyrbj6qaqzR-4XnLbvtKU84FB_JLu5x5A'
sheet_name = 'EXP_5_main_lab'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
exp5lab = pd.read_csv(url)

In [14]:
exp5lab.FEMALE = exp5lab.FEMALE.astype(str)
exp5lab.MALE = exp5lab.MALE.astype(str)
exp5lab.TEMP = exp5lab.TEMP.astype(str)

exp5lab.FERT_PERC = exp5lab.FERT_PERC.str.replace(',', '.')
exp5lab.FERT_PERC = pd.to_numeric(exp5lab.FERT_PERC)

exp5lab.SWIM_PERC = exp5lab.SWIM_PERC.str.replace(',', '.')
exp5lab.SWIM_PERC = pd.to_numeric(exp5lab.SWIM_PERC)

exp5lab['FAMILY'] = exp5lab.FEMALE + 'x' + exp5lab.MALE
exp5lab['CODE_FULL'] = exp5lab.CODE + exp5lab.CUP_NUMBER.astype(str)

In [15]:
exp5lab

,№,CODE,CUP_NUMBER,FEMALE,MALE,MUT,TEMP,TIME,DEAD,FERT,...,SHOCK_ALIVE,SHOCK_NAME,HATCHED_DEAD,HATCHED_ALIVE,HATCHED_NAME,SWIM,SWIM_NAME,SWIM_PERC,FAMILY,CODE_FULL
0,1,11CC,1,1,1,C,C,NaN,5,37,...,37,Polina,0,37,Sasha,33,Vika,89.189189,1x1,11CC1
1,2,11CC,2,1,1,C,C,NaN,2,42,...,41,Polina,1,40,Sasha,33,Vika,78.571429,1x1,11CC2
2,3,11CC,3,1,1,C,C,NaN,5,33,...,31,Polina,2,29,Sasha,26,Vika,78.787879,1x1,11CC3
3,4,11CC,4,1,1,C,C,NaN,11,38,...,37,Polina,2,35,Sasha,32,Vika,84.210526,1x1,11CC4
4,5,12CC,1,1,2,C,C,NaN,4,58,...,59,Lera,10,49,Vika,45,Polina,77.586207,1x2,12CC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,21M38,4,2,1,M,38,40.0,3,54,...,53,Alya(d)+Lera(a),3,50,Vika,33,Vika,61.111111,2x1,21M384
60,61,22M38,1,2,2,M,38,40.0,2,40,...,40,Lera,4,36,Vika,34,Polina,85.000000,2x2,22M381
61,62,22M38,2,2,2,M,38,40.0,2,43,...,40,Lera,4,36,Vika,35,Polina,81.395349,2x2,22M382
62,63,22M38,3,2,2,M,38,40.0,0,58,...,55,Lera,1,54,Vika,50,Polina,86.206897,2x2,22M383


In [7]:
exp5lab.columns

Index(['№', 'CODE', 'CUP_NUMBER', 'FEMALE', 'MALE', 'MUT', 'TEMP', 'TIME',
       'DEAD', 'FERT', 'FERT_NAME', 'TOTAL', 'FERT_PERC', 'SHOCK_DEAD',
       'SHOCK_ALIVE', 'SHOCK_NAME', 'HATCHED_DEAD', 'HATCHED_ALIVE',
       'HATCHED_NAME', 'SWIM', 'SWIM_NAME', 'SWIM_PERC', 'FAMILY'],
      dtype='object')

genotype:    
mutagen      
shock      
family???? (separate - 1 family 1 file???)      
     
      
phenotype:     
swim 1/0    
от чего считать???? фертилизация? тотал?

In [19]:
code = []
mut = []
shock = []
swim = []

for row in exp5lab.index:
    total_count = int(exp5lab.loc[row, 'TOTAL'])
    swim_count = int(exp5lab.loc[row, 'SWIM'])
    
    code += [exp5lab.loc[row, 'CODE_FULL']] * total_count
    mut += ([1] * total_count) if exp5lab.loc[row, 'MUT'] == 'M' else ([0] * total_count)
    shock += ([1] * total_count) if exp5lab.loc[row, 'TEMP'] == '38' else ([0] * total_count)
    
    swim += [1] * swim_count
    swim += [0] * (total_count - swim_count)

In [20]:
ind_data = pd.DataFrame({'CODE': code, 'MUT': mut, 'SHOCK': shock, 'SWIM': swim})

In [26]:
ind_data.reset_index(inplace=True)
ind_data

,index,CODE,MUT,SHOCK,SWIM
0,0,11CC1,0,0,1
1,1,11CC1,0,0,1
2,2,11CC1,0,0,1
3,3,11CC1,0,0,1
4,4,11CC1,0,0,1
...,...,...,...,...,...
2770,2770,22M384,1,1,0
2771,2771,22M384,1,1,0
2772,2772,22M384,1,1,0
2773,2773,22M384,1,1,0


In [ ]:
ind_data[['MUT', 'SHOCK']].to_csv('/Users/emulciber/MitoClub/fish-strong-purifying-selection/scripts/w-test/genotype.txt', sep='\t', index=False)
ind_data[['index', 'SWIM']].to_csv('/Users/emulciber/MitoClub/fish-strong-purifying-selection/scripts/w-test/phenotype.txt', sep='\t', index=False, header=False)

# FISHER TEST

## EXP 5

In [31]:
ind_data['MUT_SHOCK'] = ind_data.MUT.astype(str) + ind_data.SHOCK.astype(str)

### For all families

In [131]:
table = pd.concat([ind_data[ind_data['SWIM'] == 0].MUT_SHOCK.value_counts(), \
                   ind_data[ind_data['SWIM'] == 1].MUT_SHOCK.value_counts()], axis=1)
table.columns = ['dead', 'alive']

average = table.sum().sum() / 4

table['weight'] = round(table.alive * (average / (table.alive + table.dead)))
table

,dead,alive,weight
11,484,362,297.0
01,250,380,418.0
10,240,407,436.0
00,157,495,527.0


In [132]:
fisher_exact(np.array([[table.loc['00', 'weight'], table.loc['01', 'weight']], \
                       [table.loc['10', 'weight'], table.loc['11', 'weight']]]))

(0.8588242394978272, 0.13547826464764956)

### Family 1: 1x1

In [135]:
fam = ind_data[ind_data['CODE'].apply(lambda x: x[0:2]) == '11']

In [136]:
table = pd.concat([fam[fam['SWIM'] == 0].MUT_SHOCK.value_counts(), \
                   fam[fam['SWIM'] == 1].MUT_SHOCK.value_counts()], axis=1)
table.columns = ['dead', 'alive']

average = table.sum().sum() / 4

table['weight'] = round(table.alive * (average / (table.alive + table.dead)))
table

,dead,alive,weight
11,210,30,22.0
01,85,100,96.0
10,56,60,92.0
00,49,124,128.0


In [137]:
fisher_exact(np.array([[table.loc['00', 'weight'], table.loc['01', 'weight']], \
                       [table.loc['10', 'weight'], table.loc['11', 'weight']]]))

(0.3188405797101449, 1.3042984464976009e-05)

### Family 2: 1x2

In [138]:
fam = ind_data[ind_data['CODE'].apply(lambda x: x[0:2]) == '12']

In [139]:
table = pd.concat([fam[fam['SWIM'] == 0].MUT_SHOCK.value_counts(), \
                   fam[fam['SWIM'] == 1].MUT_SHOCK.value_counts()], axis=1)
table.columns = ['dead', 'alive']

average = table.sum().sum() / 4

table['weight'] = round(table.alive * (average / (table.alive + table.dead)))
table

,dead,alive,weight
11,100,49,52.0
01,68,63,76.0
10,60,122,106.0
00,47,121,113.0


In [140]:
fisher_exact(np.array([[table.loc['00', 'weight'], table.loc['01', 'weight']], \
                       [table.loc['10', 'weight'], table.loc['11', 'weight']]]))

(0.7293942403177756, 0.18051896203917994)

### Family 3: 2x1

In [141]:
fam = ind_data[ind_data['CODE'].apply(lambda x: x[0:2]) == '21']

In [142]:
table = pd.concat([fam[fam['SWIM'] == 0].MUT_SHOCK.value_counts(), \
                   fam[fam['SWIM'] == 1].MUT_SHOCK.value_counts()], axis=1)
table.columns = ['dead', 'alive']

average = table.sum().sum() / 4

table['weight'] = round(table.alive * (average / (table.alive + table.dead)))
table

,dead,alive,weight
11,137,122,90.0
10,71,119,120.0
01,62,95,116.0
00,47,113,135.0


In [143]:
fisher_exact(np.array([[table.loc['00', 'weight'], table.loc['01', 'weight']], \
                       [table.loc['10', 'weight'], table.loc['11', 'weight']]]))

(0.8728448275862069, 0.5105514928263923)

### Family 4: 2x2

In [144]:
fam = ind_data[ind_data['CODE'].apply(lambda x: x[0:2]) == '22']

In [145]:
table = pd.concat([fam[fam['SWIM'] == 0].MUT_SHOCK.value_counts(), \
                   fam[fam['SWIM'] == 1].MUT_SHOCK.value_counts()], axis=1)
table.columns = ['dead', 'alive']

average = table.sum().sum() / 4

table['weight'] = round(table.alive * (average / (table.alive + table.dead)))
table

,dead,alive,weight
10,53,106,111.0
11,37,161,135.0
01,35,122,129.0
00,14,137,151.0


In [146]:
fisher_exact(np.array([[table.loc['00', 'weight'], table.loc['01', 'weight']], \
                       [table.loc['10', 'weight'], table.loc['11', 'weight']]]))

(1.4236329352608423, 0.045048306362734575)

## EXP 8

In [3]:
sheet_id = '1aqHnDaiRXntyrbj6qaqzR-4XnLbvtKU84FB_JLu5x5A'
sheet_name = 'EXP_8_second_main_factory'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
exp8fac = pd.read_csv(url)

exp8fac = exp8fac.iloc[9:13,].reset_index(drop=True)

In [4]:
exp8fac[['CODE', 'SWIM']]

,CODE,SWIM
0,CC,120000.0
1,C38,87000.0
2,MC,40000.0
3,M38,8000.0


In [8]:
res = fisher_exact([[120000, 87000], \
                       [40000, 8000]])
print(res)

(0.27586206896551724, 0.0)
